### Chains with multiple inputs

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain

llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

prompt_template = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input="a parrot")

/var/folders/cc/_kcvjw313pl6kbf8c3bdtnyc0000gn/T/ipykernel_81219/1191947449.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)


{'input': 'a parrot',
 'text': 'Why did the parrot wear a raincoat?\n\nBecause it wanted to be a poly-ester!'}

In [3]:
prompt_template = PromptTemplate(input_variables=["input", "language"], template="Tell me a joke about {input} in {language}")
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke({"input": "a parrot", "language": "hindi"})

{'input': 'a parrot',
 'language': 'hindi',
 'text': 'एक आदमी ने एक तोता खरीदा। वह तोता हमेशा बातें करता था। \n\nआदमी: "तू क्या करता है?"\n\nतोता: "मैं अपने मालिक के लिए चाय बनाता हूँ!"\n\nआदमी: "अच्छा! तो फिर तुझे चाय बनाने का मजा आया?"\n\nतोता: "नहीं, चाय बनाने के बाद हर बार तुम मुझे बेकार समझते हो!" 😄'}

Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [4]:
from langchain.chains.sequential import SequentialChain

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = PromptTemplate.from_template(
    template="You ordered {dish_name} from {restaurantName} and your experience was {experience}. Write a review: "
)
chain_review = LLMChain(llm=llm, prompt=prompt_review, output_key="review")

#=============================================================================================================

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = LLMChain(llm=llm, prompt=prompt_comment, output_key="comment")

#==============================================================================================================

# This is an LLMChain to summarize a review.
prompt_summary = PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {comment}"
)
chain_summary = LLMChain(llm=llm, prompt=prompt_summary, output_key="summary")

#================================================================================================================

# This is an LLMChain to translate a summary into German.
prompt_translation = PromptTemplate.from_template(
    template="""    
    Translate the summary to 3 languages: 
    1. Hindi
    2. Spanish
    3. French
    \n\n {summary}
    """
)
chain_translation = LLMChain(
    llm=llm, prompt=prompt_translation, output_key="lang_translation"
)

#==================================================================================================================

### Sequential Chains

In [5]:
overall_chain = SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name","restaurantName","experience"],
    output_variables=["review", "comment", "summary", "lang_translation"],
)

overall_chain.invoke({"dish_name": "Chicken Biryani", "restaurantName":"Akbar Biryani", "experience": "It was awful!"})

{'dish_name': 'Chicken Biryani',
 'restaurantName': 'Akbar Biryani',
 'experience': 'It was awful!',
 'review': "**Review of Chicken Biryani from Akbar Biryani**\n\nRating: ★☆☆☆☆\n\nI recently ordered Chicken Biryani from Akbar Biryani, and I must say that my experience was nothing short of disappointing. From the moment the dish arrived, I had a sinking feeling that I was in for a letdown.\n\nFirst and foremost, the aroma, which is usually a highlight of a good biryani, was underwhelming. The spices seemed muted, and there was none of that delightful, fragrant scent that you typically expect from this classic dish. When I finally opened the container, I was greeted by a presentation that was far from appealing. The rice looked clumpy and oddly colored, lacking the vibrant hues one associates with perfectly cooked biryani.\n\nMoving on to the chicken, I was surprised at how tough and dry it was. It's fair to say that it felt more like an afterthought than the star of the dish. In addit

Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

In [6]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

positive_template = """You are an AI that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them. \
Here is the text:
{input}"""

neutral_template = """You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}"""

In [7]:
prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]

In [8]:

# prompt_infos is expected to be a list of dictionaries, each containing at least "name" and "prompt_template" keys.
# For each prompt info:
#   Extract the name and prompt template.
#   Create a PromptTemplate object that takes one input variable called "input".
#   Create an LLMChain: This is an LLM + a prompt. So, each "destination" (e.g., "food_review", "hotel_review", etc.) gets its own LLMChain.
#   Store each chain in the destination_chains dictionary, keyed by its name.

# Result: A dictionary where each key is a destination name and each value is a ready-to-use LLMChain for that intent.
    

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
destination_chains

{'positive': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an AI that focuses on the positive side of things. Whenever you analyze a text, you look for the positive aspects and highlight them. Here is the text:\n{input}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1166a6d10>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x116637290>, root_client=<openai.OpenAI object at 0x116634790>, root_async_client=<openai.AsyncOpenAI object at 0x116685210>, model_name='gpt-4o-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}),
 'neutral': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an AI that has a neutral perspective. You just provide a balanced analysis of the te

In [9]:
# For each prompt in prompt_infos, create a string like "food_review: Handles food related reviews".
# Combine all such strings into a newline-separated list (destinations_str).
# Print it for debugging. This will help the router know what all "destinations" it can send queries to.

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

positive: Good for analyzing positive sentiments
neutral: Good for analyzing neutral sentiments
negative: Good for analyzing negative sentiments


In [ ]:
# Use a predefined template (MULTI_PROMPT_ROUTER_TEMPLATE) that expects the available destinations.
# Plug in the list of destinations you just built.
# Build a PromptTemplate for the router, which will use the user input and output a routing decision.
# Set the output_parser to RouterOutputParser() (this parses the router’s decision).

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)


# Create a Router Chain that will use your LLM and the router prompt.
# When given an input, it decides which destination chain (from step 1) to route the input to.
router_chain = LLMRouterChain.from_llm(llm, router_prompt)


# MultiPromptChain is the master orchestrator.
# It uses:
# The router chain to select a destination.
# The dictionary of destination chains.
# A default chain (for "neutral" or unknown inputs).
# verbose=True gives you logs as it routes and processes.

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True,
)

chain.invoke({"input": "I ordered Pizza Salami for 9.99$ and it was awesome but awful !"})



> Entering new MultiPromptChain chain...


/var/folders/cc/_kcvjw313pl6kbf8c3bdtnyc0000gn/T/ipykernel_81219/677733521.py:26: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


positive: {'input': 'I ordered Pizza Salami for $9.99 and it was awesome!'}
> Finished chain.


{'input': 'I ordered Pizza Salami for $9.99 and it was awesome!',
 'text': "That's fantastic! It's wonderful to hear that you enjoyed your Pizza Salami so much. The price of $9.99 is quite reasonable for a delicious meal. It's great when you can treat yourself to something tasty and have a positive experience with your order!"}